In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2022-04-07 21:34:00.878766: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 21:34:00.879426: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [3]:
import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [21]:
model = Sequential()

# Convolution layer 1
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolution layer 2
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolution layer 2
model.add(Conv2D(50, (3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten to 1D before hidden layers
model.add(Flatten()) 

# Hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))


model.add(Dense(1, activation ='sigmoid'))

In [22]:
model.compile(
    loss='binary_crossentropy', 
    metrics=[tf.keras.metrics.AUC(num_thresholds=50)], 
    optimizer='adam')

In [23]:
checkpoint_filepath = '/tmp/checkpoint/melanoma_{epoch:02d}-{val_loss:.2f}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [24]:
_, counts = np.unique(y, return_counts=True)

class_weights = {
    0: counts[0] / np.max(counts),
    1: 1 / (counts[1] / np.max(counts))
}

class_weights

{0: 1.0, 1: 4.347593582887701}

In [25]:
# training the model for 10 epochs
H = model.fit(
    X_train, 
    y_train, 
    batch_size=80, 
    epochs=10,  
    class_weight=class_weights, 
    validation_data=(X_test, y_test),
    verbose=1)

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 519s 324ms/sample - loss: 1.3127 - auc: 0.5271 - val_loss: 1.1244 - val_auc: 0.5103
Epoch 2/10
1600/1600 [==============================] - 521s 326ms/sample - loss: 1.1156 - auc: 0.6018 - val_loss: 1.1086 - val_auc: 0.6647
Epoch 3/10
1600/1600 [==============================] - 521s 326ms/sample - loss: 1.0956 - auc: 0.6256 - val_loss: 1.0693 - val_auc: 0.6858
Epoch 4/10
1600/1600 [==============================] - 528s 330ms/sample - loss: 1.0864 - auc: 0.6348 - val_loss: 1.0621 - val_auc: 0.6893
Epoch 5/10
1600/1600 [==============================] - 520s 325ms/sample - loss: 1.0583 - auc: 0.6622 - val_loss: 1.0695 - val_auc: 0.6962
Epoch 6/10
1600/1600 [==============================] - 520s 325ms/sample - loss: 1.0778 - auc: 0.6503 - val_loss: 1.0732 - val_auc: 0.7052
Epoch 7/10
1600/1600 [==============================] - 518s 324ms/sample - loss: 1.0576 - auc: 0.6704 - val_loss

In [26]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%d_%m_%Y-%H_%M_%S")
model.save(f'melanoma_model_training_end_{current_time}')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-04-07 18:03:59.566784: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: melanoma_model_training_end_07_04_2022-18_03_58/assets
